# Buckles Plot

This notebook demonstrates how to create Buckles plots using GeoSuite's `buckles_plot()` function.

## Overview

Buckles plots (Bulk Volume Water vs Porosity) are used to:
- Identify productive reservoir zones
- Distinguish pay from non-pay intervals
- Assess reservoir quality using BVW cutoff

GeoSuite uses **matplotlib** with **signalplot** styling for clean, professional plots.

This notebook will show you how to:

1. Load well log data and calculate water saturation
2. Create Buckles plots with `buckles_plot()`
3. Interpret BVW cutoffs and reservoir quality
4. Customize plot parameters

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs

print("GeoSuite imported successfully!")

## 1. Load Well Log Data and Calculate Water Saturation

Load demo well log data and calculate water saturation (Sw) needed for Buckles plots.

In [ ]:
# Load demo well log data
df = load_demo_well_logs()

print(f"Loaded {len(df):,} data points")
print(f"\nAvailable columns: {df.columns.tolist()}")

# Find porosity and resistivity columns
porosity_cols = [col for col in df.columns if 'PHI' in col.upper() or 'NPHI' in col.upper() or 'POR' in col.upper()]
resistivity_cols = [col for col in df.columns if 'RT' in col.upper() or 'RES' in col.upper() or 'ILD' in col.upper()]

porosity_col = porosity_cols[0] if porosity_cols else df.columns[1]
resistivity_col = resistivity_cols[0] if resistivity_cols else df.columns[2]

print(f"\nUsing porosity column: {porosity_col}")
print(f"Using resistivity column: {resistivity_col}")

# Calculate water saturation
df['SW'] = calculate_water_saturation(
    phi=df[porosity_col].values,
    rt=df[resistivity_col].values,
    rw=0.05,  # Formation water resistivity
    m=2.0,    # Cementation exponent
    n=2.0,    # Saturation exponent
    a=1.0     # Tortuosity factor
)

print(f"\nWater saturation range: {df['SW'].min():.3f} to {df['SW'].max():.3f}")
df.head()

## 2. Create Buckles Plot

Use GeoSuite's `buckles_plot()` function to create a Buckles plot with BVW cutoff line.


In [ ]:
# Create Buckles plot with default BVW cutoff (0.04)
fig = buckles_plot(
    df=df,
    porosity_col=porosity_col,
    sw_col='SW',
    cutoff=0.04,  # BVW cutoff - typical range: 0.03-0.05
    title='Buckles Plot - Demo Well'
)

plt.show()


## 3. Customize BVW Cutoff

Adjust the BVW cutoff based on your reservoir characteristics. Typical cutoffs range from 0.03 to 0.05.


In [ ]:
# Create Buckles plot with custom BVW cutoff
# Lower cutoff (0.03) = more conservative, higher cutoff (0.05) = more optimistic
fig = buckles_plot(
    df=df,
    porosity_col=porosity_col,
    sw_col='SW',
    cutoff=0.03,  # More conservative cutoff
    title='Buckles Plot - Custom BVW Cutoff (0.03)'
)

plt.show()


## 4. Identify Reservoir Quality Zones

Use the BVW cutoff to classify reservoir quality.


In [ ]:
# Calculate Bulk Volume Water (BVW)
df['BVW'] = df[porosity_col] * df['SW']

# Classify reservoir quality based on BVW cutoff
cutoff = 0.04
df['Reservoir_Quality'] = pd.cut(
    df['BVW'],
    bins=[0, cutoff, 0.1, 1.0],
    labels=['Good (Pay)', 'Fair', 'Poor (Non-Pay)']
)

print("Reservoir Quality Distribution:")
print(df['Reservoir_Quality'].value_counts())
pay_count = df['Reservoir_Quality'].value_counts().get('Good (Pay)', 0)
print(f"\nPay zone (BVW < {cutoff}): {pay_count} points")


## 5. Color by Reservoir Quality

Color the Buckles plot points by reservoir quality classification.


In [ ]:
# Create Buckles plot colored by reservoir quality
# First, convert quality to numeric for coloring
quality_map = {'Good (Pay)': 1, 'Fair': 2, 'Poor (Non-Pay)': 3}
df['Quality_Num'] = df['Reservoir_Quality'].map(quality_map)

fig = buckles_plot(
    df=df,
    porosity_col=porosity_col,
    sw_col='SW',
    cutoff=0.04,
    color_by='Quality_Num',  # Color by reservoir quality
    title='Buckles Plot - Colored by Reservoir Quality'
)

plt.show()


## 6. Summary

This notebook demonstrated:

-  Creating Buckles plots with `buckles_plot()` (matplotlib with signalplot styling)
-  Calculating Bulk Volume Water (BVW = Porosity × Sw)
-  Using BVW cutoff to identify pay zones
-  Classifying reservoir quality zones
-  Customizing plot appearance

### Note on Plotting

GeoSuite's plotting functions use **matplotlib** with **signalplot** styling for clean, professional visualizations. The `signalplot` library provides minimalist, publication-ready styling automatically applied to all plots.

### Interpreting Buckles Plots

- **Points below BVW cutoff**: Hydrocarbon-bearing pay zones
- **Points above BVW cutoff**: Water-bearing or non-pay zones
- **Sw isolines**: Show constant water saturation contours
- **Lower BVW**: Better reservoir quality (lower water content)

### Typical BVW Cutoffs

- **0.03**: Conservative, high-quality reservoirs
- **0.04**: Standard cutoff for most sandstones
- **0.05**: More optimistic, may include transition zones

### Next Steps

- Use `geosuite.petro.pickett_plot()` for porosity-resistivity analysis
- Use `geosuite.plotting.create_strip_chart()` to visualize logs with quality zones
- Use `geosuite.ml` for facies classification to further refine reservoir zones
